In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

In [ ]:
gears_1: np.ndarray = cv2.imread('trybik.jpg')
gears_gray_1: np.ndarray = cv2.cvtColor(gears_1, cv2.COLOR_BGR2GRAY)
gears_2: np.ndarray = cv2.imread('trybiki2.jpg')
gears_gray_2: np.ndarray = cv2.cvtColor(gears_2, cv2.COLOR_BGR2GRAY)

gears_inverted_1: np.ndarray = cv2.bitwise_not(gears_gray_1)
gears_inverted_2: np.ndarray = cv2.bitwise_not(gears_gray_2)

In [ ]:
from typing import Tuple, List


def contour(image: np.ndarray) -> Tuple[List, int]:
    _, binary = cv2.threshold(image, 0, 255, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    eroded = cv2.erode(binary, np.full((3, 3), 1), iterations=1)
    
    contours, _ = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    maximum_index = max(range(len(contours)), key=lambda x: contours.__getitem__(x).shape[0])
    
    return contours, maximum_index

In [ ]:
contour_list, max_index = contour(gears_inverted_1)

plt.axis('off')
plt.imshow(
    cv2.drawContours(
        np.zeros(gears_inverted_1.shape[:2], dtype=np.uint8),
        contour_list,
        max_index,
        color=[255]
    ),
    cmap='gray'
)

In [ ]:
sobel_x = cv2.Sobel(gears_inverted_1, cv2.CV_64F, 1, 0, ksize=5)
sobel_y = cv2.Sobel(gears_inverted_1, cv2.CV_64F, 0, 1, ksize=5)

gradient_amplitude = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
gradient_amplitude /= np.amax(gradient_amplitude)
orientation = ((np.degrees(np.arctan2(sobel_y, sobel_x)) + 360) % 360).astype(np.int32)

In [ ]:
reference_point = cv2.moments(contour_list[max_index], True)

c = (int(reference_point['m10'] / reference_point['m00']), int(reference_point['m01'] / reference_point['m00']))

In [ ]:
r_table = [list() for _ in range(360)]

c_x, c_y = c

for contour in contour_list:
    for contour_x, contour_y in contour.reshape(-1, 2):
        vector = (contour_x - c_x, contour_y - c_y)
        length = np.linalg.norm(vector)
        o_x_angle = np.arctan2(vector[1], vector[0])
        
        r_table[orientation[contour_x, contour_y]].append((length, o_x_angle))

r_table

In [ ]:
sobel_x_2 = cv2.Sobel(gears_inverted_2, cv2.CV_64F, 1, 0, ksize=5)
sobel_y_2 = cv2.Sobel(gears_inverted_2, cv2.CV_64F, 0, 1, ksize=5)

gradient_amplitude_2 = np.sqrt(sobel_x_2 ** 2 + sobel_y_2 ** 2)
gradient_amplitude_2 /= np.amax(gradient_amplitude_2)

orientation_2 = ((np.degrees(np.arctan2(sobel_y_2, sobel_x_2)) + 360) % 360).astype(np.int32)

In [ ]:
image_size_x, image_size_y = gradient_amplitude_2.shape

hough_space: np.ndarray = np.zeros((image_size_x, image_size_y, 2), dtype=np.uint8)

for x_index in range(image_size_x):
    for y_index in range(image_size_y):
        if gradient_amplitude_2[x_index, y_index] <= 0.5:
            continue
        
        angle = orientation_2[x_index, y_index]
        for radius, alpha in r_table[angle]:
            x1 = int(x_index - radius * np.cos(alpha))
            y1 = int(y_index - radius * np.sin(alpha))
            
            if 0 <= x1 < image_size_x and 0 <= y1 < image_size_y:
                hough_space[x1, y1, 0] += 1
                hough_space[x1, y1, 1] = angle

In [ ]:
plt.axis('off')
plt.imshow(np.log(hough_space[:, :, 0] + 1), cmap='gray')

In [ ]:
hough_space[:, :, 0] = cv2.GaussianBlur(hough_space[:, :, 0], (13, 13), 0)

In [ ]:
top_5 = []

border: int = 75

hough_space_x, hough_space_y = hough_space[:, :, 0].shape

for _ in range(5):
    hough_max_x, hough_max_y = np.where(hough_space[:, :, 0] == hough_space[:, :, 0].max())
    hough_max_x, hough_max_y = hough_max_x[0], hough_max_y[0]
    top_5.append((hough_max_y, hough_max_x))
    hough_space[max(0, hough_max_x - border): min(hough_space_x, hough_max_x + border),
                max(0, hough_max_y - border): min(hough_space_y, hough_max_y + border),
                0] = 0

In [ ]:
top_5

In [ ]:
contours_img = cv2.imread("trybiki2.jpg")

for x, y in top_5:
    plt.plot(x, y, 'ro')
    contours_img = cv2.drawContours(contours_img, contour_list[max_index] + np.array((x, y)) - c, -1, (0, 0, 255), 1)

contours_img = cv2.cvtColor(contours_img, cv2.COLOR_BGR2RGB)
plt.axis('off')
plt.imshow(contours_img)